<a href="https://colab.research.google.com/github/arefin/z3riddles/blob/main/Cursed_Dice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Solving [this problem](https://youtu.be/urOkfsIRFlw?si=2X5Uhk5J0ni_STjW).

Note this problem is really about [Sicherman Dice](https://en.wikipedia.org/wiki/Sicherman_dice) in disguise.

In [1]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 8.5 MB/s eta 0:00:00


In [2]:
from z3 import *

In [3]:
d1 = [Int(f'd1_{i}') for i in range(6)]
d2 = [Int(f'd2_{i}') for i in range(6)]
s = Solver()
for i in range(6):
  s.add(0 < d1[i])
  s.add(0 < d2[i])
  s.add(d1[i] <= 4)

In [5]:
# Compute the number of ways to roll each sum using standard dice.
from collections import defaultdict
ways = defaultdict(int)
for i in range(1, 7):
  for j in range(1, 7):
    ways[i + j] += 1
ways

defaultdict(int,
            {2: 1,
             3: 2,
             4: 3,
             5: 4,
             6: 5,
             7: 6,
             8: 5,
             9: 4,
             10: 3,
             11: 2,
             12: 1})

In [7]:
# This is inspired by http://www.hakank.org/z3/sicherman_dice.py.
for k in ways:
  s.add(
      ways[k] == Sum(
        [If(d1[i] + d2[j] == k, 1, 0) for i in range(6) for j in range(6)]))

In [8]:
# Break symmetry.
for i in range(5):
  s.add(d1[i] <= d1[i + 1])
  s.add(d2[i] <= d2[i + 1])

In [12]:
s.check()

sat

In [13]:
m = s.model()
d1_values = [m.evaluate(d1[i]) for i in range(6)]
d2_values = [m.evaluate(d2[i]) for i in range(6)]
print(d1_values, d2_values)

[1, 2, 2, 3, 3, 4] [1, 3, 4, 5, 6, 8]


In [15]:
s.add(Or(
    [d1[i] != m[d1[i]] for i in range(6)]
    + [d2[i] != m[d2[i]] for i in range(6)]))
s.check()

unsat